## Imports and Variables

In [ ]:
import numpy as np
import pandas as pd
import itertools as it
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
import random
import time
from IPython.display import display

In [ ]:
# variables
closepairs = [['C', 'T'], ['A', 'T'], ['A', 'G']] # needed for ``close'' error mode

## Dictionaries and Frequencies
This need not be changed

In [ ]:
def make_dict(S):
    d = { }
    for s in S:
        if not s in d:
            d[s] = 0
    return d

In [ ]:
# Define character sets
othchars = {'N', '\n'} # Expected characters that should be ignored
nucchars = {'T', 'A', 'C', 'G'} # Characters that should be read as part of the RNA # , 'B', 'K', 'M', 'S', 'R', 'W', 'Y'
corenucchars = ['T', 'A', 'C', 'G'] # Subset of characters of RNA that are valid
cdnchars = [] # All triplets of nucchars
for c in it.product(nucchars, repeat=3):
    codon = c[0]+c[1]+c[2]
    cdnchars.append(codon)
corecdnchars = [] # All triplets of corenucchars
for c in it.product(corenucchars, repeat=3):
    codon = c[0]+c[1]+c[2]
    corecdnchars.append(codon)

In [ ]:
# Codon dictionary
codondict = {
    "TTT": "Phenyl-alanine",
    "TTC": "Phenyl-alanine",
    "TTA": "Leucine",
    "TTG": "Leucine",
    "TCT": "Serine",
    "TCC": "Serine",
    "TCA": "Serine",
    "TCG": "Serine",
    "TAT": "Tyrosine",
    "TAC": "Tyrosine",
    "TAA": "Stop",
    "TAG": "Stop",
    "TGT": "Cysteine",
    "TGC": "Cysteine",
    "TGA": "Stop",
    "TGG": "Tryptophan",
    "CTT": "Leucine",
    "CTC": "Leucine",
    "CTA": "Leucine",
    "CTG": "Leucine",
    "CCT": "Proline",
    "CCC": "Proline",
    "CCA": "Proline",
    "CCG": "Proline",
    "CAT": "Histidine",
    "CAC": "Histidine",
    "CAA": "Glutamine",
    "CAG": "Glutamine",
    "CGT": "Arginine",
    "CGC": "Arginine",
    "CGA": "Arginine",
    "CGG": "Arginine",
    "ATT": "Isoleucine",
    "ATC": "Isoleucine",
    "ATA": "Isoleucine",
    "ATG": "Methionine",
    "ACT": "Threonine",
    "ACC": "Threonine",
    "ACA": "Threonine",
    "ACG": "Threonine",
    "AAT": "Asparagine",
    "AAC": "Asparagine",
    "AAA": "Lysine",
    "AAG": "Lysine",
    "AGT": "Serine",
    "AGC": "Serine",
    "AGA": "Arginine",
    "AGG": "Arginine",
    "GTT": "Valine",
    "GTC": "Valine",
    "GTA": "Valine",
    "GTG": "Valine",
    "GCT": "Alanine",
    "GCC": "Alanine",
    "GCA": "Alanine",
    "GCG": "Alanine",
    "GAT": "Aspartic Acid",
    "GAC": "Aspartic Acid",
    "GAA": "Glutamic Acid",
    "GAG": "Glutamic Acid",
    "GGT": "Glycine",
    "GGC": "Glycine",
    "GGA": "Glycine",
    "GGG": "Glycine",
}
aminonames = []
for c in corecdnchars:
    if not codondict[c] in aminonames:
        aminonames.append(codondict[c])

rev_codondict = make_dict(aminonames) # dictionary from amino acid to codons
for c in corecdnchars:
    if  rev_codondict[codondict[c]] == 0:
        rev_codondict[codondict[c]] = []
    rev_codondict[codondict[c]].append(c)
        
print(aminonames)
print(rev_codondict)

In [ ]:
# load distribution of codons
distdict = {'TTT': 5379930, 'TTC': 3489330, 'TTG': 3346091, 'TTA': 2854192, 'TCT': 3713851, 'TCC': 3333211, 'TCG': 827472, 'TCA': 3560263, 'TGT': 3671777, 'TGC': 3358600, 'TGG': 4198572, 'TGA': 3896165, 'TAT': 2717421, 'TAC': 1971858, 'TAG': 1765414, 'TAA': 2772671, 'CTT': 3604102, 'CTC': 3418446, 'CTG': 4886439, 'CTA': 2028492, 'CCT': 3994829, 'CCC': 3584074, 'CCG': 1395220, 'CCA': 4126496, 'CGT': 850978, 'CGC': 1167168, 'CGG': 1393193, 'CGA': 935301, 'CAT': 3177732, 'CAC': 3021415, 'CAG': 4927410, 'CAA': 3494785, 'GTT': 2540730, 'GTC': 2064226, 'GTG': 3312170, 'GTA': 1790976, 'GCT': 3339168, 'GCC': 3427017, 'GCG': 1193087, 'GCA': 3324824, 'GGT': 2258681, 'GGC': 3176324, 'GGG': 3156393, 'GGA': 3967764, 'GAT': 2672919, 'GAC': 2444849, 'GAG': 3952791, 'GAA': 4332881, 'ATT': 3552064, 'ATC': 2458843, 'ATG': 3518926, 'ATA': 2584899, 'ACT': 2896694, 'ACC': 2744211, 'ACG': 915076, 'ACA': 3584179, 'AGT': 2959982, 'AGC': 3579694, 'AGG': 3790603, 'AGA': 4636759, 'AAT': 3574871, 'AAC': 2726839, 'AAG': 4257699, 'AAA': 5772129}

# count amino acids, and their percentage
total = 0
da = make_dict(aminonames)
dt = make_dict(aminonames)

for c in corecdnchars:
    da[codondict[c]] += distdict[c]
    total += distdict[c]
    
for c in aminonames:
    dt[c] = da[c] / total

# sort and graph
X = list(aminonames)
X.sort(reverse=True, key=lambda x : da[x])
Y = [da[x] for x in X]
    
fig, ax = plt.subplots(figsize =(16, 9))
plt.bar(X,Y,width=0.5,color=["tab:blue", "tab:orange", "tab:green", "tab:red"])
plt.title("Amino Frequency")
plt.xlabel('Aminoacid')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()
plt.close()

## General Helper Functions

In [ ]:
# empty for now
# like - really general. Helper functions for error modes should be grouped at the respective error mode cell

## Generating the RNA Sequence

In [ ]:
# returns a codon given a number between 0 and 1
def find_codon(p):
    current = 0
    curprob = 0        
    
    while current < len(aminonames):
        curprob += dt[aminonames[current]]
        if p < curprob:
            return aminonames[current]
        current += 1
        
    return aminonames[-1]

In [ ]:
# return whether there is an error given a number between 0 and 1
def has_error(p, p_err):
    if p < p_err:
        return 1
    return 0

In [ ]:
# generates input with an RNA sequence of n codons, and accompanying error, modified RNA, and error code 
def gen_RNAerr(n, p_err, errmode):
    rna = ""
    err = []
    mod_rna = ""
    err_code = []
    
    # create a sequence with n codons
    for i in range(n):
        r = random.uniform(0, 1)
        cdn = random.choice(rev_codondict[find_codon(r)])
        rna += cdn
    
    # for each nucleotide, determine whether there is an error, and compute the resulting nucleotide and error code by the error mode
    for i in range(3*n):
        r = random.uniform(0, 1)
        cur_err = has_error(r, p_err)
        err.append(cur_err)
        
        mod_char, err_bit = errmode[0](rna[i],cur_err)
        err_code.append(err_bit)
        mod_rna += mod_char

    return rna, err, mod_rna, err_code

## Reading the RNA Sequence

In [ ]:
# determines all codons we get when error-correcting codon c with error code e using error mode errmode
def pos_codons(c, e, errmode):
    p = [errmode[1](c[i], e[i]) for i in range(3)]
    return [x + y + z for x in p[0] for y in p[1] for z in p[2]]

In [ ]:
# finds whether an rna sequence can be read successfully.
# we interpret reading successfully as: for each codon, either a codon representing the original amino acid is recorded OR ALL possible codons after correction of the recorded codon represent the original amino acid
def can_read(rna, err, mod_rna, err_code, errmode):
    n = len(rna)
    if not n == len(mod_rna):
        print("Invalid length")
    
    output = True
    nsucc_err = 0 # number of characters that can be individually corrected successfully
    ntot_err = 0
    nsucc = [0 for i in range(4)] # nsucc[i] = number of successful reads with i errors in codon
    ntot = [0 for i in range(4)] # nsucc[i] = number of codons with i errors
    
    for i in range(math.floor(n/3)):
        # get information for the current codon
        real_codon = rna[3*i : 3*i + 3]
        mod_codon = mod_rna[3*i : 3*i + 3]
        err_codon = err[3*i : 3*i + 3]
        err_code_codon = err_code[3*i : 3*i + 3]
        
        
        # compute success for errors in current codon
        for i in range(3):
            if err_codon[i] == 0:
                continue
            ntot_err += 1
            if real_codon[i] == mod_codon[i]:
                nsucc_err += 1
                continue
            corr_codons = errmode[1](mod_codon[i], err_code_codon[i])
            
            valid = True
            for c in corr_codons:
                if not c == real_codon[i]:
                    valid = False
                    break
            
            if valid:
                nsucc_err += 1
        
        # compute number of errors in current codon
        n_err = 0
        for e in err_codon:
            if e == 1:
                n_err += 1
        
        ntot[n_err] += 1
        
        ident_codons = rev_codondict[codondict[real_codon]] # find all codons representing same aminoacid as real codon
        
        if mod_codon in ident_codons: # the output of the machine corresponds to the same codon
            nsucc[n_err] += 1
            continue
        
        # check if all possible codons after error correction are correct
        errcorr_codons = pos_codons(mod_codon, err_code_codon, errmode)
        valid = True
        for c in errcorr_codons:
            if not c in ident_codons:
                valid = False
                break
        
        if valid:
            nsucc[n_err] += 1
            continue
        
        output = False
    
    return output, nsucc_err, nsucc, ntot_err, ntot

## Doing Experiments

In [ ]:
# does a single experiment on a number of trials
def do_experiment(codoncount, p_err, expcount, errmode):
    successes = 0
    succ_err_read = 0
    tot_err_read = 0
    succ_read = [0 for i in range(4)]
    tot_read = [0 for i in range(4)]
    
    for i in range(expcount):
        rna, err, mod_rna, err_code = gen_RNAerr(codoncount, p_err, errmode)
        read, succ_err, succ, tot_err, tot = can_read(rna, err, mod_rna, err_code, errmode)
        if read:
            successes += 1
            
        succ_err_read += succ_err
        tot_err_read += tot_err
        
        for i in range(4):
            succ_read[i] += succ[i]
            tot_read[i] += tot[i]
        
    return successes, succ_err_read, succ_read, tot_err_read, tot_read

In [ ]:
# does an experiment for all specified sequence lenghts (in codons), and error probabilities
def do_experiments(codoncounts, p_errs, expcounts, errmode):
    out = [[ 0 for j in p_errs ] for i in codoncounts]
    
    for i in range(len(codoncounts)):
        for j in range(len(p_errs)):
            res = do_experiment(codoncounts[i], p_errs[j], expcounts, errmode)
            out[i][j] = res
    
    return out

In [ ]:
# returns several figues:
# - heatmap
# - combined line plot with all fixed lengths 
# - combined line plot with all fixed p_errs
# - list of plots, one for each fixed length, in order
# - list of plots, one for each fixed p_err, in order
# parameters are as input and output from the experiments
# pass an empty string as title to have the title as only the statistic used
# statistic chooses the heatmap to create:
# =0 : probability of success
# =1 : probability correct reading for error
# =2 : probability correct codon no errors
# =3 : probability correct codon one error
# =4 : probability correct codon two errors
# =5 : probability correct codon three errors
# =6 : probability correct codon any errors
def drawfigures_experiments(codoncounts, p_errs, expcount, data_in, statistic=0, title="Placeholder", show = True, save=False):
    #print("Title:", title)
    #print("Figures shown:", show)
    #print("Figures saved:", save)
    
    wbar = min([p_errs[i + 1] - p_errs[i] for i in range(len(p_errs)) if i+1 < len(p_errs)])
    
    if statistic < 0 or statistic > 6: # if incorrectly specified, set to default
        statistic = 0
    
    title_full = title
    
    if not title_full == "":
        title_full += ": "
        
    if statistic == 0:
        title_full += "P_Succ"
    if statistic == 1:
        title_full += "P_SuccCor Single Error"
    if statistic == 2:
        title_full += "P_SuccCor Codon No Errors"
    if statistic == 3:
        title_full += "P_SuccCor Codon One Error"
    if statistic == 4:
        title_full += "P_SuccCor Codon Two Errors"
    if statistic == 5:
        title_full += "P_SuccCor Codon Three Errors"
    if statistic == 6:
        title_full += "P_SuccCor Codon Any Errors"
    
    data = np.zeros((len(codoncounts), len(p_errs)))
    for i in range(len(codoncounts)):
        for j in range(len(p_errs)):
            if statistic == 0: # probability success
                if expcount == 0:
                    data[i][j] = 1
                    continue
                data[i][j] = data_in[i][j][0] / expcount
            if statistic == 1: # probability correct reading for error
                if data_in[i][j][3] == 0:
                    data[i][j] = 1
                    continue
                data[i][j] = data_in[i][j][1] / data_in[i][j][3]
            if statistic == 2: # probability correct codon no errors
                if data_in[i][j][4][0] == 0:
                    data[i][j] = 1
                    continue
                data[i][j] = data_in[i][j][2][0] / data_in[i][j][4][0]
            if statistic == 3: # probability correct codon one error
                if data_in[i][j][4][1] == 0:
                    data[i][j] = 1
                    continue
                data[i][j] = data_in[i][j][2][1] / data_in[i][j][4][1]
            if statistic == 4: # probability correct codon two errors
                if data_in[i][j][4][2] == 0:
                    data[i][j] = 1
                    continue
                data[i][j] = data_in[i][j][2][2] / data_in[i][j][4][2]
            if statistic == 5: # probability correct codon three errors
                if data_in[i][j][4][3] == 0:
                    data[i][j] = 1
                    continue
                data[i][j] = data_in[i][j][2][3] / data_in[i][j][4][3]
            if statistic == 6: # probability correct codon any errors
                if data_in[i][j][4][0] + data_in[i][j][4][1] + data_in[i][j][4][2] + data_in[i][j][4][3] == 0:
                    data[i][j] = 1
                    continue
                data[i][j] = (data_in[i][j][2][0] + data_in[i][j][2][1] + data_in[i][j][2][2] + data_in[i][j][2][3]) / (data_in[i][j][4][0] + data_in[i][j][4][1] + data_in[i][j][4][2] + data_in[i][j][4][3])
                
    figcnt = 0
    figheat = plt.figure(figcnt, figsize=(7.5, 5.625))
    ax = figheat.add_axes(sns.heatmap(data, xticklabels=p_errs, yticklabels=codoncounts, linewidth=1, annot=True, vmin=0, vmax=1))
    plt.title(title_full)
    plt.xlabel('p_err')
    plt.ylabel('#Codons')
    if save: plt.savefig("output/heatfig/" + str(statistic) + "_heat_" + title + ".pdf")
    if show: plt.show()
    plt.close()
    figcnt += 1
    
    figcodonline = plt.figure(figcnt, figsize=(7.5, 5.625))
    ax = plt.axes()
    for i in range(len(codoncounts)):
        ax.plot(p_errs, data[i], label=str(codoncounts[i]))
    plt.legend(title = "#Codons", loc="upper right")
    plt.title(title_full)
    plt.xlabel('p_err')
    plt.ylabel('Probability')
    plt.xlim([0, 1])
    if save: plt.savefig("output/codonfig/" + str(statistic) + "_line_combined_" + title + ".pdf")
    if show: plt.show()
    plt.close()
    figcnt += 1
    
    figperrline = plt.figure(figcnt, figsize=(7.5, 5.625))
    ax = plt.axes()
    for i in range(len(p_errs)):
        ax.plot(codoncounts, data[:,i], label=str(p_errs[i]))
    plt.legend(title = "p_err", loc="upper right")
    plt.title(title_full)
    plt.xlabel('#Codons')
    plt.ylabel('Probability')
    plt.xlim([0, codoncounts[-1] + 1])
    if save: plt.savefig("output/perrfig/" + str(statistic) + "_line_combined_" + title + ".pdf")
    if show: plt.show()
    plt.close()
    figcnt += 1
    
    figcods = [[-1, -1] for i in range(len(codoncounts))]
    figerrs = [[-1, -1] for i in range(len(p_errs))]

    for i in range(len(codoncounts)):
        fig = plt.figure(figcnt, figsize=(7.5, 5.625))
        ax = plt.axes()
        ax.plot(p_errs, data[i])
        plt.title(title_full + ", #Codons: " + str(codoncounts[i]))
        plt.xlabel('p_err')
        plt.ylabel('Probability')
        plt.xlim([0, 1])
        figcods[i][0] = fig
        if save: plt.savefig("output/codonfig/" + str(statistic) + "_line_codons" + str(codoncounts[i]) + "_" + title + ".pdf")
        if show: plt.show()
        plt.close()
        figcnt += 1
        
        fig = plt.figure(figcnt, figsize=(7.5, 5.625))
        ax = plt.axes()
        ax.bar(p_errs, data[i],width=wbar, color=["tab:blue", "tab:orange", "tab:green", "tab:red"])
        plt.title(title_full + ", #Codons: " + str(codoncounts[i]))
        plt.xlabel('p_err')
        plt.ylabel('Probability')
        plt.xlim([0, 1])
        figcods[i][1] = fig
        if save: plt.savefig("output/codonfig/" + str(statistic) + "_bar_codons" + str(codoncounts[i]) + "_" + title + ".pdf")
        if show: plt.show()
        plt.close()
        figcnt += 1

    for i in range(len(p_errs)):
        fig = plt.figure(figcnt, figsize=(7.5, 5.625))
        ax = plt.axes()
        ax.plot(codoncounts, data[:,i])
        plt.title(title_full + ", p_err: " + str(p_errs[i]))
        plt.xlabel('#Codons')
        plt.ylabel('Probability')
        plt.xlim([0, codoncounts[-1] + 1])
        figerrs[i][0] = fig
        if save: plt.savefig("output/perrfig/" + str(statistic) + "_line_perr" + str(p_errs[i]) + "_" + title + ".pdf")
        if show: plt.show()
        plt.close()
        figcnt += 1
        
        fig = plt.figure(figcnt, figsize=(7.5, 5.625))
        ax = plt.axes()
        ax.bar(codoncounts, data[:,i],width=15/codoncounts[-1], color=["tab:blue", "tab:orange", "tab:green", "tab:red"])
        plt.title(title_full + ", p_err: " + str(p_errs[i]))
        plt.xlabel('#Codons')
        plt.ylabel('Probability')
        plt.xlim([0, codoncounts[-1] + 1])
        figerrs[i][1] = fig
        if save: plt.savefig("output/perrfig/" + str(statistic) + "_bar_perr" + str(p_errs[i]) + "_" + title + ".pdf")
        if show: plt.show()
        plt.close()
        figcnt += 1
    
    return figheat, figcodonline, figperrline, figcods, figerrs

## Error Modes (or Error Models)
Defining the error modes
An error mode is a tuple of two functions:
- one for generating the error code and (modified character) from the character and error
- another for getting a list of nucleotides that the error gets corrected to

Current modes:
- 01 : binary error code, 0 bit represents no error, 1 bit represents error (correct is other nucleotide than read RNA)
- 100 : fully recoverable error code (stores sequence in error code), read RNA is uniform random per nucleotide
- close : binary error code, 0 bit represents a nucleotide close to the correct one (or the correct one) was read, 1 bit represents a far away nucleotide to the correct one was read
- real :  real-valued error code in range [0,1], value represents certainty of correctness (chosen arbitrarily in [0,1) in case of an error) where higher represents more certain of correctness

In [ ]:
# error bit 0 represents certain no error, error bit 1 represents certain error
def errgen_01(char, err):
    if err == 0:
        return char, 0
    return random.choice([x for x in corenucchars if not x == char]), 1

def errcor_01(char, err):
    if err == 0:
        return [char]
    
    return [x for x in corenucchars if not x == char]

em_01 = (errgen_01, errcor_01)

In [ ]:
# error bit represents the correct nucleotide, 
def errgen_100(char, err):
    return random.choice([x for x in corenucchars]), char

def errcor_100(char, err):
    return [err]    

em_100 = (errgen_100, errcor_100)

In [ ]:
# error bit 0 represents close nucleotide to original read, error bit 1 represents far nucleotide to original read
def close_char(char):
    close = set()
    for p in closepairs:
        if char in p:
            close.update(p)
    return close

def errgen_close(char, err):
    if err == 0:
        return char, 0
    
    close = close_char(char)
    mod_char = random.choice([x for x in corenucchars if not x == char])
    
    if mod_char in close:
        return mod_char, 0
    
    return mod_char, 1
    
def errcor_close(char, err):
    close = close_char(char)
    if err == 0:
        return close
    return [x for x in corenucchars if not x in close]

em_close = (errgen_close, errcor_close)

In [ ]:
# error bit (in [0,1]) represents how `confident' we are (higher is better)
def errgen_real(char, err):
    if err == 0:
        return char, 1
    return random.choice([x for x in corenucchars if not x == char]), random.random()

def errcor_real(char, err):
    aux = random.random()
    if aux <= err:
        return [char]
    return [random.choice([x for x in corenucchars if not x == char])]

em_real = (errgen_real, errcor_real)

## Finally, we call these functions

In [ ]:
# This is how the RNA is generated for one experiment
rna, err, mod_rna, err_code = gen_RNAerr(10, 0.3, em_close)
print(rna)
print(''.join(map(str,err)))
print(mod_rna)
print(''.join(map(str,err_code)))

print("")

# This is how the RNA is generated for one experiment
rna, err, mod_rna, err_code = gen_RNAerr(10, 0.3, em_close)
print(rna)
print(''.join(map(str,err)))
print(mod_rna)
print(''.join(map(str,err_code)))

In [ ]:
# This is how an experiment is performed
# it returns the:
# - number of successes (full sequence successfully read
# - number of errors successfully corrected
# - number of successful codon reads given the number of errors per codon [0, 1, 2, 3 errors]
# - number of total errors
# - number of total codon reads given the number of errors per codon [0, 1, 2, 3 errors]
do_experiment(10, 0.35, 5000, em_close) # (sequence length, error probability, number of trials, error mode)

In [ ]:
# This is how we do multiple experiments (for several sequence lengths and error probabilities)
# it returns a 2D list of the individual experiment results
res = do_experiments([10,20], [0.01,0.1], 8196, em_real) # ([sequence lenghts], [error probabilities], nmber of trials, error mode)
res

In [ ]:
# For saving: set save=True and make sure that in the directory of this notebook there exists the folders output>heatfig, output>codonfig, output>perrfig
heat, codonlines, perrslines, codon, perrs = drawfigures_experiments([10,20], [0.01,0.1], 8196, res, statistic=0, title = "Yadayada", show=True, save=False)

## Number of Trials
To determine a good number of trials to choose, we run a small set of experiments for several number of trials.
We then pick a satisfactory one.

## Experiments
We are now equipped to write the actual experiments.
The number of trials is set to 2048 = 2^12.

In [ ]:
# this is how you save only one figure 
h = plt.figure(heat)
#plt.savefig("tmp.pdf") (UNCOMMENT THIS)

In [ ]:
codoncounts = [10, 50, 100, 500, 1000, 1500, 2000]
machineerrors = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
errormode = em_01
trials = 2048

res = do_experiments(codoncounts, machineerrors, trials, errormode)

In [ ]:
heat, codline, errline, figcods, figerrs = drawfigures_experiments(codoncounts, machineerrors, trials, res, statistic=0, title = "Binary", show=False, save=False)

In [ ]:
tmp = plt.figure(heat)